In [30]:
import pandas as pd
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import config
input_file = "egp_pilot.json"
samples_per_level = 2
examples_per_batch = 10

In [36]:
egp = pd.read_csv('../dat/egponline.csv', index_col=0)
egp_samples = egp.groupby(['Level', 'type'], group_keys=False).apply(lambda x: x.sample(min(len(x), samples_per_level), random_state=config.SEED))
egp_samples['Example'] = egp_samples['Example'].str.replace(r"\(.*\)", "", regex=True).str.strip()

def get_prompt(construction):
    lexical_range = ''
    if not pd.isna(construction["Lexical Range"]):
        if construction["Lexical Range"] == 1:
            lexical_range = 'low'
        elif construction["Lexical Range"] == 2:
            lexical_range = 'medium'
        elif construction["Lexical Range"] == 3:
            lexical_range = 'high'
        lexical_range = f'Use words of {lexical_range} difficulty in the rule.'
    return f'Learn the grammar rule "{construction["Can-do statement"]}" ({construction["SuperCategory"]}, {construction["SubCategory"]}, {construction["guideword"]}). It is CEFR level {construction["Level"]}. {lexical_range}\nExamples:\n{construction["Example"]}\nCreate {examples_per_batch} more examples using that rule.'

egp_samples['prompt'] = egp_samples.apply(get_prompt, axis=1)
egp_samples['augmented_examples'] = [[] for _ in range(len(egp_samples))]
egp_samples['augmented_examples_source'] = [[] for _ in range(len(egp_samples))]
egp_samples['augmented_negative_examples'] = [[] for _ in range(len(egp_samples))]
egp_samples['augmented_negative_examples_source'] = [[] for _ in range(len(egp_samples))]

In [37]:
egp_previous = pd.read_json('../dat/' + input_file)

for index, row in egp_previous.iterrows():
        egp_samples.at[index, 'augmented_examples'] = row['augmented_examples']
        egp_samples.at[index, 'augmented_examples_source'] = row['augmented_examples_source']
        egp_samples.at[index, 'augmented_negative_examples'] = row['augmented_negative_examples']
        egp_samples.at[index, 'augmented_negative_examples_source'] = row['augmented_negative_examples_source']

In [38]:
egp_samples

,#,SuperCategory,SubCategory,Level,Lexical Range,guideword,Can-do statement,Example,type,prompt,augmented_examples,augmented_examples_source,augmented_negative_examples,augmented_negative_examples_source
646,647,NOUNS,noun phrases,A1,1.0,FORM: DETERMINER + NOUN,Can form simple noun phrases with a limited ra...,There is also a park near my house. \n\nMy hus...,FORM,"Learn the grammar rule ""Can form simple noun p...","[He has a dog in his backyard., She buys a new...","[170934, 170934, 170934, 170934, 170934, 17093...","[He has it in the backyard., She buys them eve...","[170934, 170934, 170934, 170934, 170934, 17093..."
110,111,ADVERBS,adverbs and adverb phrases: types and meanings,A1,NaN,FORM: 'VERY' + ADJECTIVES,Can use 'very' to modify common gradable adjec...,My home is very nice. \n\nMy teacher is very g...,FORM,"Learn the grammar rule ""Can use 'very' to modi...",[],[],[],[]
239,240,CLAUSES,subordinated,A1,NaN,"FORM/USE: 'BECAUSE', REASONS",Can use a finite subordinate clause with 'beca...,I love her because she is friendly. \n\nI like...,FORM/USE,"Learn the grammar rule ""Can use a finite subor...","[He is happy because the weather is sunny., Th...","[428211, 428211, 428211, 428211, 428211, 42821...","[He is happy; the weather is sunny., They pass...","[428211, 428211, 428211, 428211, 428211, 42821..."
287,288,DETERMINERS,articles,A1,1.0,FORM/USE: NO ARTICLE,Can use no article before a limited range of s...,In my free time I usually play football. \n\nI...,FORM/USE,"Learn the grammar rule ""Can use no article bef...",[],[],[],[]
299,300,DETERMINERS,demonstratives,A1,NaN,"USE: 'THIS', FUTURE",Can use 'this' with time and date words to ref...,… this summer I want to spend my holiday in my...,USE,"Learn the grammar rule ""Can use 'this' with ti...","[Let's meet this Friday, it works best for me....","[745390, 745390, 745390, 745390, 745390, 74539...","[Let's meet next Friday, it works best for me....","[745390, 745390, 745390, 745390, 745390, 74539..."
851,852,PRESENT,present continuous,A1,1.0,USE: EVENTS IN PROGRESS,Can use the present continuous with a limited ...,[in a message to someone who is ill] We are mi...,USE,"Learn the grammar rule ""Can use the present co...",[],[],[],[]
384,385,FUTURE,future with will and shall,A2,NaN,FORM: NEGATIVE 'WILL',Can use the negative forms 'will not' and 'won...,Without them I will not be able to play footba...,FORM,"Learn the grammar rule ""Can use the negative f...",[We will not finish the project on time if we ...,"[575959, 575959, 575959, 575959, 575959, 57595...",[We will finish the project on time if we star...,"[575959, 575959, 575959, 575959, 575959, 57595..."
65,66,ADJECTIVES,superlatives,A2,NaN,FORM: WITH '-ST',Can form superlative adjectives using the suff...,I always watch the latest music videos from th...,FORM,"Learn the grammar rule ""Can form superlative a...",[],[],[],[]
1106,1107,CLAUSES,conditional,A2,NaN,"FORM/USE: PRESENT SIMPLE 'IF' CLAUSE, REAL CON...",Can use 'if' + present simple with present sim...,If I have spare time I always read a book. \n\...,FORM/USE,"Learn the grammar rule ""Can use 'if' + present...","[If it rains, we can stay indoors and play gam...","[114906, 114906, 114906, 114906, 114906, 11490...","[When it rains, staying indoors and playing ga...","[114906, 114906, 114906, 114906, 114906, 11490..."
59,60,ADJECTIVES,superlatives,A2,NaN,FORM/USE: WITH 'OF' + NOUN,Can use a limited range of prepositional phras...,My teacher said that it was the best project o...,FORM/USE,"Learn the grammar rule ""Can use a limited rang...",[],[],[],[]


In [29]:
egp_samples.to_json('../dat/' + input_file)